# House Price Prediction

## Data Preprocessing and feature engineering

**Author** - `Prasad Parjane`

Source -(https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data?select=train.csv)

